In [9]:
import h5py
#import h5py_cache as h5c
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torchdata
import torch.multiprocessing
import sys

#torch.multiprocessing.set_start_method('spawn')

from hdf5_dataset import H5Dataset
from network import Net, NetCCFFF



test_set = H5Dataset('/home/wuxr/graspping/gpd-master/pytorch/gpd_orgin/data/for_test.h5')
test_loader = torchdata.DataLoader(test_set, batch_size=64, shuffle=True)

train_set = H5Dataset('/home/wuxr/graspping/gpd-master/pytorch/gpd_orgin/data/for_train.h5')
#train_loader = torchdata.DataLoader(dset, batch_size=64, shuffle=True, num_workers=2)
train_loader = torchdata.DataLoader(train_set, batch_size=64, shuffle=True)

dataiter = iter(train_loader)
images, labels = dataiter.next()

# Use GPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)



Loaded images of shape torch.Size([6894, 60, 60, 12]), type torch.uint8, and labels of shape torch.Size([6894, 1]), type torch.int32.
Loaded images of shape torch.Size([29999, 60, 60, 12]), type torch.uint8, and labels of shape torch.Size([29999, 1]), type torch.int32.
cuda:0


In [18]:
import torchvision
import matplotlib.pyplot as plt
import numpy as np
print(images.shape)
LR = 0.001
CHANNELS = [20, 50, 500]
class Net(nn.Module):
    def __init__(self, input_channels):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, CHANNELS[0], 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(CHANNELS[0], CHANNELS[1], 5)
        self.fc1 = nn.Linear(CHANNELS[1] * 12 * 12, CHANNELS[2])
        self.fc2 = nn.Linear(CHANNELS[2], 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, x.shape[1] * x.shape[2] * x.shape[3])
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net=Net(12)

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=0.0005)

optimizer = optim.Adam(net.parameters(), lr=LR, weight_decay=0.0005)

#加入tensorboard设置。
from torch.utils.tensorboard import SummaryWriter
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/log')
#写入tensorbard中。
writer.add_graph(net, images)
writer.close()

torch.Size([64, 12, 60, 60])
